# The Pst class

The `pst_handler` module contains the `Pst` class for dealing with pest control files.  It relies heavily on `pandas` to deal with tabular sections, such as parameters, observations, and prior information.  

In [ ]:
from __future__ import print_function
import os
import numpy as np
import pyemu
from pyemu import Pst

We need to pass the name of a pest control file to instantiate:

In [ ]:
pst_name = os.path.join("henry","pest.pst")
p = Pst(pst_name)

All of the relevant parts of the pest control file are attributes of the `pst` class with the same name:



In [ ]:
p.parameter_data.head()

In [ ]:
p.observation_data.head()

In [ ]:
p.prior_information.head()

A residual file (`.rei` or `res`) can also be passed to the `resfile` argument at instantiation to enable some simple residual analysis and weight adjustments.  If the residual file is in the same directory as the pest control file and has the same base name, it will be accessed automatically:


In [ ]:
p.res

The `pst` class has some `@decorated` convenience methods related to the residuals:

In [ ]:
print(p.phi,p.phi_components)

Some additional `@decorated` convenience methods:

In [ ]:
print(p.npar,p.nobs,p.nprior)

In [ ]:
print(p.par_groups,p.obs_groups)

In [ ]:
print(type(p.par_names)) # all parameter names
print(type(p.adj_par_names)) # adjustable parameter names
print(type(p.obs_names)) # all observation names
print(type(p.nnz_obs_names)) # non-zero weight observations

The "control_data" section of the pest control file is accessible in the `Pst.control_data` attribute:

In [ ]:
print('jacupdate = {0}'.format(p.control_data.jacupdate))
print('numlam = {0}'.format(p.control_data.numlam))
p.control_data.numlam = 100
print('numlam has been changed to --> {0}'.format(p.control_data.numlam))

The `Pst` class also exposes a method to get a new `Pst` instance with a subset of parameters and or observations.  Note this method does not propagate prior information to the new instance:

In [ ]:
pnew = p.get(p.par_names[:10],p.obs_names[-10:])
print(pnew.prior_information)

You can also write a pest control file with altered parameters, observations, and/or prior information:

In [ ]:
pnew.write("test.pst")

Some other methods in `Pst` include:

In [ ]:
# add preferred value regularization with weights proportional to parameter bounds
pyemu.utils.helpers.zero_order_tikhonov(pnew)
pnew.prior_information

In [ ]:
# add preferred value regularization with unity weights
pyemu.utils.helpers.zero_order_tikhonov(pnew,parbounds=False)
pnew.prior_information

Some more `res` functionality

In [ ]:
# adjust observation weights to account for residual phi components
#pnew = p.get()
print(p.phi, p.nnz_obs, p.phi_components)
p.adjust_weights_discrepancy()
print(p.phi, p.nnz_obs, p.phi_components)

adjust observation weights by an arbitrary amount by groups:

In [ ]:
print(p.phi, p.nnz_obs, p.phi_components)
grp_dict = {"head":100}
p.adjust_weights(obsgrp_dict=grp_dict)
print(p.phi, p.nnz_obs, p.phi_components)

adjust observation weights by an arbitrary amount by individual observations:

In [ ]:
print(p.phi, p.nnz_obs, p.phi_components)
obs_dict = {"h_obs01_1":25}
p.adjust_weights(obs_dict=obs_dict)
print(p.phi, p.nnz_obs, p.phi_components)

setup weights inversely proportional to the observation values

In [ ]:
p.adjust_weights_discrepancy()
print(p.phi, p.nnz_obs, p.phi_components)
p.proportional_weights(fraction_stdev=0.1,wmax=20.0)
print(p.phi, p.nnz_obs, p.phi_components)